# Исследование надёжности заёмщиков

Необходимо подготовить проект для кредитного отдела банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.  
Результаты проекта будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
Отсутствие оглавления.
<br>1. Для чего нужно: 1 – это некий план согласно которого ты выполняешь задачу, 2 – проверяющему или пользователю легче ориентироваться в работе.
<br>2. Оглавление должно быть интерактивным – нажимаем –> переходим к тому или иному пункту решения поставленной задачи.
<br>3. Отсутствует описание данных.
</div>

## Загрузка данных и изучение общей информации по проекту

In [1]:
import pandas as pd

data = pd.read_csv('data.csv')

In [2]:
data.sample(15)

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21378,21378,0,401256.745090,59,среднее,1,вдовец / вдова,2,F,пенсионер,0,204103.704870,операции со своей недвижимостью
12150,12150,0,-2167.873499,48,среднее,1,женат / замужем,0,F,компаньон,0,148235.139062,автомобили
2581,2581,0,NaN,56,высшее,0,в разводе,3,F,госслужащий,0,NaN,строительство недвижимости
17004,17004,2,-248.561425,27,среднее,1,женат / замужем,0,M,сотрудник,0,161800.813560,образование
10105,10105,0,-4408.189939,58,среднее,1,женат / замужем,0,F,сотрудник,0,155395.006618,операции со своей недвижимостью
18173,18173,0,344186.984083,56,среднее,1,женат / замужем,0,F,пенсионер,0,81886.000355,строительство собственной недвижимости
484,484,1,-1101.890185,45,среднее,1,женат / замужем,0,M,сотрудник,0,167580.842430,автомобили
20827,20827,1,-1264.290386,42,среднее,1,женат / замужем,0,F,сотрудник,0,133987.017989,недвижимость
8140,8140,1,333028.640100,61,среднее,1,женат / замужем,0,F,пенсионер,0,128279.673929,покупка жилья для сдачи
14925,14925,0,368945.757693,62,среднее,1,гражданский брак,1,F,пенсионер,0,144149.728877,сыграть свадьбу


In [3]:
data.describe()

,Unnamed: 0,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,10762.000000,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,6213.876608,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,5381.000000,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,10762.000000,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,16143.000000,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,21524.000000,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


С помощью описательной статистики мы выявили следующие недостатки данных: минимальный возраст 0, минимальное количество детей -1, максимальное количество детей 20, данные из столбца days_employed не репрезетантивны (содержат, в том числе, и отрицательные значения), данные из столбца total_income следует преобразовать к целочисленному типу для удобства чтения.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. sample/head/tail, describe, info лучше все в одной ячейке и используй display.
<br>2. Когда делаешь выводы, старайся делать и предположения почему так произошло или какова причина. Почему 20 детей? Многодетная семья, сомнительно. Скорее всего ошиблись при вводе. Опять же можно посмотреть, это единичное значение или нет.
Проверяющему будет проще проследить ход мыслей и поправить/подсказать.
<br>3. Что за столбец Unnamed: 0. Перед загрузкай всегда смотри файл с помощью текстового редактора, чтобы иметь представление что ты загружаешь, тогда при загрузке можешь указать типы для явных данных (только если уверен на 99%) или указать столбец с индексом.
</div>

Чтобы посмотреть общую информацию по датафрейму и всем признакам, используем метод info:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


Воспользовавшись методом info можно заметить, что количество не пустых значений в столбцах 'days_employed' и 'total_income' отличается от общего количества значений в других столбцах.

In [5]:
#Подсчитываем количество пропущенных значений
data.isnull().sum()

Unnamed: 0             0
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. В ячейках кода только пояснение к коду. Никаких комментариев по выполнению работы в ячейках с кдом не должно быть, для этого есть Markdown ячейки.
</div>

Общее количество пропущенных значений в столбцах 'days_employed' и 'total_income' равно 2174, что составляет более 10% от общего количества данных в соответсвующих категориях. Так как удаление такого большого объема данных может повлиять на дальнейший анализ, необходимо произвести замену пустых значений в данных столбцах.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Из практики - 10% это не много, если данные однообразны (необходимо обосновать) и их больше ~100 000, то можно пожертвовать, если ограничено время на исследование данных. Но если стоит задача поиска аномалий - категорически нельзя удалять. Но лучше конечно пропуски заполнять.
</div>

## Предобработка данных

Более подроное изученим данные в каждом столбце таблицы по отдельности. Проверим строки в которых имеется хотя бы одно пустое значение

In [6]:
data[data.isnull().any(axis=1)].sample(5)

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7299,7299,0,NaN,31,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,на покупку подержанного автомобиля
19514,19514,0,NaN,66,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,строительство недвижимости
16204,16204,0,NaN,56,среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку автомобиля
14130,14130,0,NaN,44,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,NaN,свой автомобиль
19570,19570,0,NaN,23,среднее,1,гражданский брак,1,M,сотрудник,0,NaN,на покупку автомобиля


Проверка строк с пропусками показывает взаимосвязь между трудовым стажем и месячным доходом (данные в соответствующих столбцах отсутствуют в одних и тех же строках). Возможно это связано с тем, что клиентом не предоставлена справка о доходах в которой обычно отражается период работы в организации.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Неплохое предположение.
</div>

In [7]:
data[(data.days_employed >= 0)]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [9]:
data.income_type.value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Выборочная проверка данных позволила выдвинуть теорию о том, что трудовой стаж находится в зависимости от типа занятости. Дальнейшая проверка показала, что только две катогории имеют положительно значение в столбце days_employed: пенсионеры и безработные, т.е. люди которые на момент предоставления информация не работают, остальные категории имеют отрицательное значение и пустое. В рамках текущего проекта обработать данные по столбцу days_employed и привести их к репрезентативному виду не представляется возможным и необходимым (так как эти данные не влияют на исследование). Можно предположить, что некорректные данные появились во время выгрузки данных, но при этом сохранена определенная зависимость. Таким образом при заполнении данных необходимо учитывать тип занятости.

In [10]:
categories = data['income_type'].unique()
for i in categories:
    data.loc[(data['days_employed'].isna()) & (data['income_type'] == i), 'days_employed'] \
    = data.loc[data['income_type'] == i, 'days_employed'].median()
    data.loc[(data['total_income'].isna()) & (data['income_type'] == i), 'total_income'] \
    = data.loc[data['income_type'] == i, 'total_income'].median()

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Заполнение медианным значением в зависимости от категорий - простой, но достаточно эффективный способ. Однако у тебя зависисмость только от категорий одного критерия, можно сделать больее сложную, вложенную структуру. Есть еще более эффективные способы. Но на текущий момент это самый оптимальный.
<br>2. !!!! Придерживайся принципа - действие + отображение в ОДНОЙ ячейке.
</div>

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     21525 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      21525 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


Проверка показывает, что пропущенные значения отсутствуют.

In [12]:
data.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [13]:
data[(data.children == 20) & (data.dob_years <= 25)]['dob_years'].value_counts().sum()

5

Проверка данных по стабцу children показывает, что cтроки содержащие значение от '6' до '19' отсутствуют, что косвенно свидетельствует на ошибку ввода. Цифра 20 в категории 'Количество детей' была указана ошибочно, так как такие данные были указаны в том числе по отношении к клиентам, которые не могли иметь такое количество детей в силу возраста. Скорее всего данные были не правильно занесены в базу и вместо '20' должно быть указано '2', а вместо '-1' - '1'. При этом количество строк, содержащих значение '-1' составляет меньше процента от количества семей с одним ребенком, поэтому такие данные допустимо удалить. Частотность значени '20' составляет 3,7% от количество семей с 2 детьми, поэтому такие значения целесообразно изменить на '2'. 

In [14]:
data = data[data['children'] != -1]
data.loc[data['children'] == 20, 'children'] = 2
data.children.value_counts()

0    14149
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Я бы тоже так сделал исходя из моего комментария выше. Тут хорошо.
</div>

In [15]:
data[(data.dob_years <= 18)]['dob_years'].value_counts()

0    101
Name: dob_years, dtype: int64

In [16]:
data[(data.dob_years == 0)]

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19829,19829,0,-1574.202821,0,среднее,1,женат / замужем,0,F,сотрудник,0,142594.396847,жилье
20462,20462,0,338734.868540,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20577,20577,0,331741.271455,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21179,21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


В столбце dob_years значение '0' встречается 101 раз, при этом какие-либо закономерности, которые могут влиять на такое значение не выявлены, т.е. такие данные могут быть случайны. Так как общее количество строк, содержащих такое значение не велико (менее 0,5%), то такие строки не могут повлиять на дальнейшее исследование и могут быть удалены.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Как я говорил, удаляем, если данные однообразны, но это надо обосновать. Иначе, допустим, все представители одной категории некого критерия оказались в той выборке, которую ты удалил. Следовательно ты не сможешь описать, сделать выводы и т.д. по этой категории, а клиент не зная ничего о этой категории, может потерять выгоду.
</div>

In [17]:
data = data[data['dob_years'] != 0]
data.dob_years.describe()

count    21377.000000
mean        43.499509
std         12.248988
min         19.000000
25%         33.000000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

In [18]:
data['education'].value_counts()

среднее                13660
высшее                  4678
СРЕДНЕЕ                  766
Среднее                  706
неоконченное высшее      665
ВЫСШЕЕ                   272
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Изучение данных стоблца education показывает наличие дубликатов: имеются одинаковые категории с разным регистром. Чтобы учесть такие дубликаты, все символы в строке приводим к нижнему регистру.

In [92]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15132
высшее                  5216
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Проверка показывает, что после обработки дубликаты в столбце education отсутствуют.

In [20]:
data['gender'].value_counts()

F      14129
M       7247
XNA        1
Name: gender, dtype: int64

Проверка данных в столбце gender показывает наличие неиндефицируемого значения 'XNA'. С учетом того, что такое значение единично его можно было бы удалить, однако такое значение можно и заменить на другое (на исследование это не повлияет). Так как значение категориальное и представлено всего двумя видами данных будет целесообразным заменить его на самое распространенное, т.е. на значение моды.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>В таких случаях обычно описывают категории (F и M), и сравнивают с неизвестным значением или можно воспользоваться машинным обучением (задачи классификации). Поскольку здесь одно значение, можно сделаить так, как ты сделал, или удалить.
</div>

In [21]:
data.gender.mode()

0    F
Name: gender, dtype: object

In [22]:
#Заменяем некорректные данные на моду

data.loc[data['gender'] == 'XNA', 'gender'] = 'F'
data['gender'].value_counts(dropna=False)

F    14130
M     7247
Name: gender, dtype: int64

In [23]:
data['family_status'].value_counts(dropna=False)

женат / замужем          12302
гражданский брак          4151
Не женат / не замужем     2792
в разводе                 1181
вдовец / вдова             951
Name: family_status, dtype: int64

В столбце family_status особых замечание по хранящемся в нем значений нет. Дубликатов и пропусков данных нет.

In [24]:
data.debt.value_counts(dropna=False)

0    19645
1     1732
Name: debt, dtype: int64

Столбец debt представлен двумя значениеми: 0, который судя по всему обозначает 'False' (т.е. отсутвие долга по кредиту) и 1 -'True' (т.е. имеется долг по кредиту).

## Заменяем типы данных

Для дальнейшего удобстав представления данных поменяем из тип в столбце total_income.

In [25]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21377 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21377 non-null  int64  
 1   children          21377 non-null  int64  
 2   days_employed     21377 non-null  float64
 3   dob_years         21377 non-null  int64  
 4   education         21377 non-null  object 
 5   education_id      21377 non-null  int64  
 6   family_status     21377 non-null  object 
 7   family_status_id  21377 non-null  int64  
 8   gender            21377 non-null  object 
 9   income_type       21377 non-null  object 
 10  debt              21377 non-null  int64  
 11  total_income      21377 non-null  int32  
 12  purpose           21377 non-null  object 
dtypes: float64(1), int32(1), int64(6), object(5)
memory usage: 2.2+ MB


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Есть типы данных int32, float32 и другие. Посмотри возможные варианты. Выполни небольшой анализ по зависимости memory usage от типов данных. 
</div>

## Лемматизация данных

In [27]:
data['purpose'].value_counts(dropna=False)

свадьба                                   792
на проведение свадьбы                     772
сыграть свадьбу                           768
операции с недвижимостью                  671
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   650
операции с коммерческой недвижимостью     648
операции с жильем                         646
покупка жилья для семьи                   640
жилье                                     640
покупка жилья                             638
строительство собственной недвижимости    632
недвижимость                              629
операции со своей недвижимостью           627
строительство недвижимости                620
строительство жилой недвижимости          620
покупка своего жилья                      618
покупка недвижимости                      618
ремонт жилью                              609
покупка жилой недвижимости                602
на покупку своего автомобиля              501
заняться высшим образованием      

Изучение столбца purpose показавает большое количество одних и тех же категорий записанных по разному. Для обработки таких данных проведем над ними лемматизацию.

In [93]:
from pymystem3 import Mystem
m = Mystem()

text = ' '.join(data['purpose'].unique())
lemmas = m.lemmatize(text)

На основе списка лемматизированных слов осуществим категоризацию данных: появятся 4 категории, которые будут охватывать все ранее представленные.

In [94]:
def purpose_group(lemmas):
    for word in lemmas.split(' '): 
        if 'недвижимост' in word or 'жил' in word:
            return 'недвижимость'
        if 'авто' in word:
            return 'автомобиль'
        if 'свадьб' in word:
            return 'свадьба'      
    return 'образование'

data['purpose'] = data['purpose'].apply(purpose_group)
data['purpose'].value_counts(dropna=False)

недвижимость    10769
автомобиль       4282
образование      3994
свадьба          2332
Name: purpose, dtype: int64

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Тут все верно. 
</div>

In [30]:
data

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,недвижимость
21521,21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,автомобиль
21522,21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,автомобиль


После всех проведенных манипуляций с данными проведем проверку на дубликаты с помощью метода .duplicated()

In [32]:
data.duplicated().sum()

0

Дубликатов не выявлено.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Если days_employed не информативен, то его стоило сразу удалить после сделанного вывода.
</div>

## Анализ данных

In [95]:
#Функция для отображения необходимых данных в сводной таблице
def pivot_table(data, index):
    data_pivot = data.pivot_table(index=index, values='debt', aggfunc=['count', 'sum'])
    data_pivot.columns = ['total_amount', 'debt']
    data_pivot['debt %'] = data_pivot['debt'] * 100 / data_pivot['total_amount']
    data_pivot = data_pivot.fillna(0)
    return data_pivot

In [96]:
pivot_table(data, 'children')

,total_amount,debt,debt %
children,,,
0,14080,1058,7.514205
1,4802,441,9.183673
2,2117,202,9.541804
3,328,27,8.231707
4,41,4,9.756098
5,9,0,0.000000


Наблюдается прямая зависимость между наличием детей и погашением кредитных обязательств в срок: клиенты без детей более ответсвенно относятся к выплатам по кредиту.

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Я бы не сказал, что клиенты без детей более "ответсвенно" относятся к выплатам по кредиту, я бы сказал что они меньше подвержены рискам, связанным с наличем детей.
</div>

In [70]:
pivot_table(data, 'family_status')

,total_amount,debt,%
family_status,,,
Не женат / не замужем,2792,273,9.777937
в разводе,1181,85,7.197290
вдовец / вдова,951,62,6.519453
гражданский брак,4151,386,9.298964
женат / замужем,12302,926,7.527231


Наблюдается прямая зависимость между наличием или отсутствием супруга/супруги (в том числе если если клиент ранее был в браке) и выплатой кредита в срок. При этом наиболее дисциплинироваными являются вдовцы.

Для определения зависимости между уровнем доходов и наличием задолженности по кредиту проведем категоризацию по уровню даходу. Разабьем данные на 3 примерно равные группы со средним доходом, ниже и выше среднего.

In [89]:
data['income_level'] = pd.qcut(data['total_income'],
                              q=3,
                              labels=['ниже среднего', 'средний', 'выше среднего'])

In [90]:
pivot_table(data, 'income_level')

,total_amount,debt,%
income_level,,,
ниже среднего,7126,578,8.111142
средний,7279,628,8.627559
выше среднего,6972,526,7.544464


In [ ]:
Выборка показывает, что клиенты с более высоким уровнем дохода наиболее дисциплинированы

<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>Аналогично с комментарием выше. "Дисциплинированы" лучше не употреблять. Я бы написал так. У кого выше доход, у тех больше возможностей погасить кредит. Также обычно такие люди более финасово грамотны и умеют распоряжатся своими деньгами.
</div>

In [72]:
pivot_table(data, 'purpose')

,total_amount,debt,%
purpose,,,
автомобиль,4282,399,9.318076
недвижимость,10769,779,7.233726
образование,3994,370,9.263896
свадьба,2332,184,7.890223


Согласно имеющихся данных наиболее дисциплинированы клиенты, которые берут кредит на приобретение недвижимости или организацию свадьбы. 

## Вывод

В ходе выполнения проекта были решены следующие задачи. 

При предобработке данных были выявлены пустые значения в категориях 'трудовой стаж' и 'доход в месяц', которые могли быть не внесены изначально в базу из-за не предоставления соответсвующих сведений клиентом. Такие данные были заменены медианным значением. С помощью ручного поиска с учетом регистра были выявлены дубликаты в категории 'образование', что свидетельствует о некорректном вводе данных в базу. Все данные в указанной категории были приведены к одному регистру. В столбце total_income с помощью метода .astype() был изменен тип данных на целый для лучшей удобства работы. Лемматизацию данных столбца purpose провели с помощью библиотеки pymystem3. Для этого столбца сделали категоризацию, т.к. изначально одни и теже категории были записаны разными словами. Кроме того, была проведана категоризация по уровню дохода, чтобы в последствии применить новые категории в анализе зависимости между уровнем дохода и возвратом кредита.

Анализ предоставленных данных показал, что вдовец без детей, но с высоким уровнем дохода, берущий кредит на организацию свадьбы, наиболее предпочтителен для кредитной организации.)


<div style="border:solid green 2px; padding: 20px"> <h1 style="color:green; margin-bottom:20px">Комментарий </h1>
<br>1. Выводы всегда и везде выводы делай в виде списка. А так "каша" получается.
<br>2. Выводы по работе офрмляй так, чтобы это было понятно, что это выводы. В том числе и по тексту. Выводы имеют свой стиль формления, комментарии по работе свой. Но не такое офрмление, что выколи глаз (не надо разноцветных текстов и т.д.). Используй списки, Bold, Italic и т.д.
<br>3. Выводы старайся делать после кадого этапа (этапы согласно оглавлению), кратко что сделал и основные выводы. В конце работы проделанные этапы и ключевые выводы.
<br>4. Про вдавца исправь на неженатый)   
<br>5. Мои комментарии не удаляй!!!
</div>